# Прогнозирование температуры звезды

## Введение

Закзачик — обсерватория «Небо на ладони» — хочет перейти на технологии машинного обучения для определения температуры поверхности звезд. 

Для поиска решения на основе нейронных сетей заказчик предоставил харасктеристики и значения температуры 240 звезд.

Заказчик ожидает, что разработанная модель будет иметь RMSE не более 4500.

Заказчик просит включит в отчет график «Факт — Прогноз», где по горизонтальной оси будут отложены условные номера звёзд, а по вертикальной — температура в Кельвинах.

### Цель проекта

Целью проекта является разработка архитектуры и обучение модели, предсказывающей температуру поверхности звезды. 

Качество модели оценивается метрикой среднеквадратичная ошибка (RMSE) и должно быть меньше или равно 4500.

### Основные этапы

В проекты выполнены следующие основные этапы:
- исследовательский анализ данных,
- разработка базовой модели,
- улучшение модели.

### Выводы

Мы разработали модель, позволяющую определять температуру звезды по 5 характеристикам. Среднквадратичная ошибка предсказания на тестовых данных 2000.



## Подготовка кода

### Загрузка библиотек

### Функции и классы

## Загрузка данных

## Исследовательский анализ данных

## Подготовка данных

## Построение baseline модели

## Улучшение модели

## Выводы